Egune Task for finetuning TTS models using Mongolian Dataset
> Add blockquote



In [1]:
!sudo apt-get install python3.9 python3.9-distutils python3.9-dev -y
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 1
!curl -sS https://bootstrap.pypa.io/get-pip.py | python3.9

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpython3.9 libpython3.9-dev libpython3.9-minimal libpython3.9-stdlib
  python3.9-lib2to3 python3.9-minimal
Suggested packages:
  python3.9-venv binfmt-support
The following NEW packages will be installed:
  libpython3.9 libpython3.9-dev libpython3.9-minimal libpython3.9-stdlib
  python3.9 python3.9-dev python3.9-distutils python3.9-lib2to3
  python3.9-minimal
0 upgraded, 9 newly installed, 0 to remove and 38 not upgraded.
Need to get 12.2 MB of archives.
After this operation, 46.6 MB of additional disk space will be used.
Get:1 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 libpython3.9-minimal amd64 3.9.24-1+jammy1 [838 kB]
Get:2 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 python3.9-minimal amd64 3.9.24-1+jammy1 [2,075 kB]
Get:3 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubunt

In [2]:
!python3.9 -m pip install --upgrade pip setuptools wheel

In [3]:
#Check runtime on colab jupter notebook
!nvidia-smi

Fri Oct 17 07:09:18 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   48C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
#Pull github
!git clone https://github.com/dgduksict/tts-egune.git
%cd tts-egune

Cloning into 'tts-egune'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 51 (delta 23), reused 21 (delta 6), pack-reused 0 (from 0)
Receiving objects: 100% (51/51), 47.21 KiB | 1.10 MiB/s, done.
Resolving deltas: 100% (23/23), done.
/content/tts-egune


In [5]:
#Download needed packages
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 131.8 MB/s  0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 89.0 MB/s  0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 122.6 MB/s  0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 119.7 MB/s  0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 38.0 MB/s  0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 126.0 MB/s  0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of librosa to determine which version is compatible with other requirements. T

In [7]:
#Import necessary libs
from huggingface_hub import notebook_login, hf_hub_download
from datasets import load_dataset, Audio, get_dataset_config_names
import tarfile, os, pandas as pd

In [8]:
#Hugging face login

notebook_login()

In [9]:
#Define necessary functions
def example():
  return "hello duke"

example()

'hello duke'

In [ ]:
# dataset = load_dataset("mozilla-foundation/common_voice_17_0", "mn", split="train")
# save_dir = "/commonvoice_mn"
# os.makedirs(save_dir, exist_ok=True)

# Extract all tar contents (audio clips + TSV)
# for i, sample in enumerate(dataset):
#     file_path = sample["__key__"]
#     url = sample["__url__"]
#     if url.endswith(".tar") or url.endswith(".tar.gz"):
#         archive_path = os.path.join(save_dir, os.path.basename(url))
#         dataset.download_and_prepare()
#         with tarfile.open(archive_path, "r:*") as tar:
#             tar.extractall(save_dir)
#         break

# print("Extracted dataset to:", save_dir)
# !ls {save_dir}

# print(dataset.features)
# print("Len: ", len(dataset))

In [10]:
dataset = load_dataset("dgduksict/commonvoice-mn", split="train")
save_dir = "/commonvoice_mn"
os.makedirs(save_dir, exist_ok=True)

print(dataset.features)
print("Len: ", len(dataset))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


train.tsv: 0.00B [00:00, ?B/s]

dev.tsv: 0.00B [00:00, ?B/s]

test.tsv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/2191 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1894 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1933 [00:00<?, ? examples/s]

{'client_id': Value('string'), 'path': Value('string'), 'sentence_id': Value('string'), 'sentence': Value('string'), 'sentence_domain': Value('string'), 'up_votes': Value('int64'), 'down_votes': Value('int64'), 'age': Value('string'), 'gender': Value('string'), 'accents': Value('string'), 'variant': Value('float64'), 'locale': Value('string'), 'segment': Value('float64')}
Len:  2191


In [11]:
# Calculate the number of rows for edited size
edited_size = len(dataset) // 1

# Select the first half of the dataset
dataset = dataset.select(range(edited_size))

print(dataset)

Dataset({
    features: ['client_id', 'path', 'sentence_id', 'sentence', 'sentence_domain', 'up_votes', 'down_votes', 'age', 'gender', 'accents', 'variant', 'locale', 'segment'],
    num_rows: 2191
})


In [ ]:
!ls